# Iterator
Iterators are a type of object in Python containing other types of data (iterables) and can traverse all values using the method `__next__` or the function `next()`.  They can be defined using the function `iter()`, and they also can be the result of other functions found in Python.

The biggest difference between an iterator with an iterable (like a list, tuple, dictionary, or sets) is that the latter is just a container.

Let's define one iterator from a simple list:

In [ ]:
my_list = [1, 2, 3, 4, 5, 6]

my_iterator = iter(my_list)

You can traverse through all the value only using the function `next()`. You can see this when you run the following cell multiple times. Notice how the element returned will change every time you call `next`.

In [ ]:
next(my_iterator)
# or using my_iterator.__next__()

This is what a `for` statement does. It creates an iterator from an iterable and executes the method `__next__` for each loop.

You can convert an iterator in an interable using the proper function. For example, in the following code, we will transform our iterator into a set:

In [ ]:
set(my_iterator)

Keep in mind that only the remaining elements in our iterator will be shown in our previous set.

One advantage of iterators is that they only store one value at a time in memory. This is especially valuable when you design custom iterators using rules to define the next value instead of just giving a list of elements. Check the differences in the size of our list and our iterator.

In [ ]:
from sys import getsizeof

print(getsizeof(my_list))
print(getsizeof(my_iterator))

## Combinatorics in Python

Python also has some iterators that simplify some combinatoric tasks such as combinations, permutations, or cartesian products. These came in a Python module called `itertools`.

This module contains various methods that allow us to produce some particular type of iterators. Imagine you want to get all possible combinations of elements within a given list. You can do it manually using a loop inside another loop to get all possible possibilities. 

In [ ]:
import itertools
result = itertools.combinations(["a","b","c"], 2)

We can iterate over each of the elements in results in the same way we did for our simple iterator. Run the following cell multiple times to see it.

In [ ]:
result.__next__()

Let's create a single function that allows us to use four methods included in this module.

You can achieve this goal in multiple ways; that is the beauty of coding -- diversity. Some codes are more efficient regarding memory, while others speed. In the following function, we used a dictionary to hold our operations.

In [ ]:
# A suggestion for Function 1.1.1.A. 
import itertools
def combinatoric(list1,
                 list2=[],
                 length=None,
                 operation="co" 
                ):
    """Function to apply some combinatoric operation using
    predefined iterators included in itertools module.
    
    Parameters
    ----------
    list1 : iterable
        List of element to pass as iterable to iterators.
    list2 : iterable, optional
        Optional iist of element to pass as iterable only for produc iterators.
    length : int
        Desired length of each iterator result. By default it is the length 
        of list1.
    operation : str
        String that control which iterator apply. It should be one of the 
        following: 
            "co": combination. Default option
            "cr": combination with replacement
            "pe": permutation
            "pr": product
        
    Returns
    -------
    list
        Return a list with the results of a given combinatoric iterator.
    """
    
    # define default length
    if not length:
        length = len(list1)
        
    # define dictionary with different functions
    main_dict = {"co": itertools.combinations(list1, length),
                 "cr": itertools.combinations_with_replacement(list1, length),
                 "pe": itertools.permutations(list1, length),
                 "pr": itertools.product(list1, list2),
    }
    
    # exe
    result = main_dict[operation]
    
    return result
    

Test the function:

In [ ]:
combinatoric("abcd", length=2, operation="co")

Let's see the content of this iterator:

In [ ]:
list(combinatoric("abcd", "xyz", operation="pr"))

# Tree enumeration
As we saw in Chapter 3 of Inferring Phylogenies (Felsenstein, 2004), calculating the number of total possible trees, given a number of tips, has attracted the attention of mathematicians since the XIX century.

Having a clear image of how many trees we can have in specific scenarios can help us understand the problem it constitutes and the importance of some algorithms we will learn during this course.

## Number of possible places where a new tip can be added

Considering that we have a tree with `n_tips` we can calculate the maximum possible places where a new tip could be inserted using the simple formula $2n-3$. 

For example

In [ ]:
n_tips = 3 # defines size of current tree (in number of tips)
n = n_tips + 1 # calculates the number of tips that we want in our final tree (just one at the time)
n_places = 2 * n - 3 # calculates the possible number of places where we can put our tip in our n-1 tree

n_places

We can transform this into a function to reuse this code as much as we want.

In [ ]:
# A suggestion for Function 1.1.2.A
def calculate_nplaces(n_tips: int) -> int:
    """Calculate the number of possible places in a n_tips tree to put an 
    additional tip.
    """
    n = n_tips + 1 # number of tips that we want in our final tree
    n_places = 2 * n - 3 # possible number of places where we can put our tip in our n_tips tree
    return n_places

In [ ]:
print(calculate_nplaces(2))
print(calculate_nplaces(n_tips=3))

Following Felsenstein's logic, the total number of branches in our tree represents the total places where a new tip can be inserted. Using this argument, in Toytree, we can count the number of branches that a given tree has and return this number.

Let's generate a random tree in Toytree and count the branches.

In [ ]:
import toytree # imports the module toytree

random_tree = toytree.rtree.rtree(ntips=3) # calls the method rtree(), which is inside the submodule rtree (part of toytree)
random_tree.draw(); # draws our randomly generated tree

We can use the method `get_edges()` to get a `DataFrame` with all the edges in our tree. Let's put it in a variable:

In [ ]:
all_edges = random_tree.get_edges()

Now we can just count the number of edges we have. This can be done using the function `len()` (native in Python) or the attribute `shape` part of the `DataFrame` type. The attribute `shape` is a tuple of 2 elements, the number of rows and columns, respectively.

In [ ]:
all_edges.shape[0] # alternative using shape attribute 
len(all_edges) # alternative using len() function

Note the number of places is different from what we had before; this is because our tree has not an explicit branch in the root. So, we should add another extra edge to our count. 

We can put it in a function to calculate it easily:

In [ ]:
def calculate_nplaces_toytree(n_tips: int) -> int:
    """Calculate the number of possible places in a n_tips tree to put an 
    additional tip using Toytree.
    """
    random_tree = toytree.rtree.rtree(ntips=n_tips) 
    all_edges = random_tree.get_edges()
    result = len(all_edges) + 1 # do not forget add one extra branch that represent our root
    return result

Now compare this new result using Toytree with the result in our function calculate_n_places().

In [ ]:
n_tips = 3
calculate_nplaces(n_tips) == calculate_nplaces_toytree(n_tips)

## Number of trees of a given number of tips

### Rooted trees

As discussed in Felsenstein (2004), there are a couple of ways to get the possible number of bifurcating rooted trees considering the places where a *n*-tip can be added. 

Considering one of the methods mentioned, we can create a list of successive odds (places where a new tip can be added) and use it to get the number of possible tips.

We can take advantage of the function we previously declared and use it to get our list of odd numbers in the following way:

In [ ]:
ntips = 4

successive_odds = [] # To save our list of odd sucessive odd integers we can declare an empty list

# Creates a list of successive odds considering the size of the tree using our previous function
for i in range(1, ntips):
    successive_odds.append(calculate_nplaces(i))
    
total_possible_trees = 1  
for o in successive_odds:  
    total_possible_trees *= o  

total_possible_trees, successive_odds

We can put this in a function:

In [ ]:
# A suggestion for Function 1.1.2.B
def calculate_max_bifurcating_trees(ntips: int) -> tuple:
    """Calculates the maximum number of trees given a number of tips"""
    
    successive_odds = [] # To save our list of odd sucessive odd integers we can declare an empty list

    # Creates a list of successive odds considering the size of the tree using our previous function
    for i in range(2, ntips):
        successive_odds.append(calculate_nplaces(i))

    total_possible_trees = 1  
    for o in successive_odds:  
        total_possible_trees *= o  

    # return the result and the successive list of possible places
    return (total_possible_trees, successive_odds)

# Test the function getting the number of trees with 50 tips
calculate_max_bifurcating_trees(50)[0]

In [ ]:
# Using f-strings we can easily format big numbers with scientific notation
f"The number of possible rooted trees having 50 tips is: {calculate_max_bifurcating_trees(50)[0]:e}"

Alternativelly, we can calculate the total number of trees using the formula:
$$
\frac{\left( 2n - 3 \right)!}{2^{n - 2}\left(n - 2 \right)!}
$$

<div class="alert alert-block alert-info">
    <b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Info</b><br>
    <p style="color: black">
        Note that the formula in some editions of Felsenstein (2004) does not produce the expected result. Instead you can use the formula from Felsenstein (1978).
    </p>
<div>

In [ ]:
import math
def n_trees(n_tips):
    """Calculate the number of bifucated rooted trees using one formula"""
    den = math.factorial(2*n_tips - 3)
    num = 2**(n_tips-2) * math.factorial(n_tips - 2)
    return int(den / num)

n_trees(50)

### Unrooted trees

As Felsenstein (2004) described, we can calculate in the same way as we did in for rooted trees the number of possible places and the maximum number of trees considering that our tree is unrooted. In this case, $2n - 5$ is the maximum possible places where a new tip could be inserted in an unrooted tree.

So, let's update our previously defined function to add this capability.

We can add a new parameter to both functions that indicate if we are considering a rooted or unrooted tree. We can state rooted as default value. Because both functions are communicated, we could explicitly add this parameter in our second function; in other words, add in `calculate_max_bifurcating_trees()` the parameter rooted. 

In [ ]:
def calculate_nplaces(n_tips: int, 
                      rooted: bool = True) -> int:
    """Calculate the number of possible places in a n_tips tree to put an 
    additional tip.
    """
    
    # determine the factor in the n_places calculation
    # the same result can be achieved, reducing by one the number of tips when it unrooted
    if rooted:
        factor = 3
    else:
        factor = 5
    
    n = n_tips + 1 # number of tips that we want in our final tree
    n_places = 2 * n - factor # possible number of places where we can put our tip in our n_tips tree
    return n_places


def calculate_max_bifurcating_trees(ntips: int,
                       rooted: bool = True) -> tuple:
    """Calculates the maximum number of trees given a number of tips"""
    
    successive_odds = [] # To save our list of odd sucessive odd integers we can declare an empty list

    # Creates a list of successive odds considering the size of the tree using our previous function
    for i in range(2, ntips):
        successive_odds.append(calculate_nplaces(i, rooted))

    total_possible_trees = 1  
    for o in successive_odds:  
        total_possible_trees *= o  

    return (total_possible_trees, successive_odds)

We can test this function by calculating the number of unrooted trees possible for 10 tips:

In [ ]:
calculate_max_bifurcating_trees(10, rooted=False)[0]

Now compare the previous result with the same calculation, but now for rooted trees

In [ ]:
calculate_max_bifurcating_trees(10)[0]

You can see how the number of unrooted trees is smaller than rooted trees, but the maximum possible number of trees is still large enough. In the following test, the number is so large that Python refuses to operate with it.

In [ ]:
print("Tips\t# rooted trees\t# unr. trees\tRatio (r/unr)") # print headings

# Calculate the number 
for i in range(1,200,10):
    t_rooted = calculate_max_bifurcating_trees(i)[0]
    t_unrooted = calculate_max_bifurcating_trees(i, False)[0]
    print(i, "{:e}".format(t_rooted), "{:e}".format(t_unrooted), (t_rooted/t_unrooted), sep="\t")

## Why the number of possible trees is important?

Imagine that we can compute, evaluate, and process 1,000,000,000 trees in only one ns. How long can we take to explore all the possible topologies having 30 tips tree? Let's calculate it!

In [ ]:
# Calculating the total nanoseconds we need
nanoseconds = calculate_max_bifurcating_trees(30)[0] / 1e+9

ns_in_h = 3.6e+12  # 1 hour = 3.6E+12 ns

# Calculates hours
hours = nanoseconds / ns_in_h

print(f"{hours} h")

It looks like the number is quite big to comprehend it, let's convert it to years

In [ ]:
h_in_y = 8760 # 1 year = 8760 h

# Calculates years
years = hours / h_in_y
print(f"{years:e} years")

This number is still so big. To put some context, the estimated age of the universe is about **1.37e+10 years**. Check in the following code how long Python takes to create a list of 1 billion zeros. In other words, exploring all possible trees if we have 30 tips is technologically impossible. 

In [ ]:
%%timeit
[0 for i in range(int(1e+9))]

## Number of multifurcating trees (rooted and unrooted)

The numbers we saw previously are not the maximum possible, considering other factors. For example, when we have a multifurcating tree, a tree of 4 tips with only two nodes, the number of possible trees will increase considerably. 

Enumerating this kind of tree is a little more complex than perfectly bifurcated trees, and we should consider the number of internal nodes that can also be variable (e.g., four-tips trees with 3, 2, and 1 node).

Let's put the Felsenstein (2004) logic in a Python function and explore the results.

In [ ]:
import numpy as np
def calculate_max_multifurcating_trees(n_tips: int, 
                                       n_internal_nodes: int = None, 
                                       rooted: bool = True):
    """Calculate the number of possible of multifurcating trees given  
    a number of tips.
    """
        
    # reduce one possible tip in unrooted trees
    if not rooted:
        n_tips = n_tips - 1
    
    # calculate n_nodes to create matrix
    n_nodes = n_tips - 1
    
    # create an empty numpy array to store the progressive results
    matrix_ntrees = np.zeros(shape=(n_nodes + 1, n_tips + 1))
   
    # iterate over each n and each m
    for m in range(1, n_nodes+1):
        for n in range(2, n_tips+1):

            # fill diagonal that is equal to the bifurcating trees
            if m == n - 1:
                matrix_ntrees[m, n] = calculate_max_bifurcating_trees(n)[0]

            # if it is a different m than n-1, fill the multifurcating trees counts
            else:
                # m = 1 is an especial case because the number of trees is always 1
                if m == 1:
                    matrix_ntrees[m,n] = 1

                # if not m =1 them apply the following formula
                else: 
                    matrix_ntrees[m,n] = (n + m - 2) * matrix_ntrees[m-1, n-1] + m * matrix_ntrees[m, n-1]
    
    # if n_internal_nodes is passed get position in matrix that store n_tips and n_internal_nodes and store it as int
    if n_internal_nodes:
        result = int(matrix_ntrees[n_internal_nodes, n_tips])
    else:
        # get max multifurcating trees
        result = int(np.sum(matrix_ntrees[:, n_tips]))
    
    return result, matrix_ntrees

We can see the expected number of trees considering multifurcation for a 5 tips rooted tree.

In [ ]:
calculate_max_multifurcating_trees(n_tips=5)

Or unrooted tree

In [ ]:
calculate_max_multifurcating_trees(n_tips=5, rooted=False)

Let's compare the numbers with the bifurcating trees for 10 tips:

In [ ]:
print(calculate_max_bifurcating_trees(10)[0])
print(calculate_max_multifurcating_trees(n_tips=10)[0])